In [ ]:
import os
from pathlib import Path
from einops import rearrange
import numpy as np

datasets = Path("/grand/insitu/cohanlon/datasets/")
raw_datasets = datasets / "raw"

partition_size = 128
partitions = datasets / "partitions"

# Make volumetric dataset partitions
for filename in raw_datasets.iterdir():
    if not filename.name.endswith(".raw"):
        raise ValueError(f"Unexpected file format: {filename}")
    metadata = filename.name[: -len(".raw")].split("_")

    name = "_".join(metadata[:-2])
    if name != "lobster":
        continue
    shape = tuple(int(dim) for dim in metadata[-2].split("x"))
    dtype = np.dtype(metadata[-1])
    print(f"Processing {name} with shape {shape} and dtype {dtype}...")

    arr = np.fromfile(filename, dtype=dtype).reshape(shape, order="F")
    partition_id = 0
    for i in range(0, shape[0], partition_size):
        for j in range(0, shape[1], partition_size):
            for k in range(0, shape[2], partition_size):
                partition_shape = (
                    min(partition_size, shape[0] - i),
                    min(partition_size, shape[1] - j),
                    min(partition_size, shape[2] - k),
                )
                partition = arr[
                    i : i + partition_shape[0],
                    j : j + partition_shape[1],
                    k : k + partition_shape[2],
                ]
                fname = f"{name}_{'x'.join([str(s) for s in partition_shape])}_{dtype}_id{partition_id}.raw"
                partition_id += 1
                partition_filename = partitions / fname
                partition.tofile(partition_filename)
                
    print("Done.")

Processing lobster with shape (301, 324, 56) and dtype uint8
Partition: (0-128, 0-128, 0-56)
Writing partition to /grand/insitu/cohanlon/datasets/partitions/lobster_128x128x56_uint8_id0.raw

Partition: (0-128, 128-256, 0-56)
Writing partition to /grand/insitu/cohanlon/datasets/partitions/lobster_128x128x56_uint8_id1.raw

Partition: (0-128, 256-324, 0-56)
Writing partition to /grand/insitu/cohanlon/datasets/partitions/lobster_128x68x56_uint8_id2.raw

Partition: (128-256, 0-128, 0-56)
Writing partition to /grand/insitu/cohanlon/datasets/partitions/lobster_128x128x56_uint8_id3.raw

Partition: (128-256, 128-256, 0-56)
Writing partition to /grand/insitu/cohanlon/datasets/partitions/lobster_128x128x56_uint8_id4.raw

Partition: (128-256, 256-324, 0-56)
Writing partition to /grand/insitu/cohanlon/datasets/partitions/lobster_128x68x56_uint8_id5.raw

Partition: (256-301, 0-128, 0-56)
Writing partition to /grand/insitu/cohanlon/datasets/partitions/lobster_45x128x56_uint8_id6.raw

Partition: (256-

In [ ]:
import os

